<a href="https://colab.research.google.com/github/Jorge-Alvarez-23/Data-Science-III_Coder/blob/main/Proy_Final_AlvarezJorge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importacion


In [2]:
import pandas as pd
import requests
from io import StringIO
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
----------------------------------------------
import chardet
import requests
import csv

# Lectura

In [3]:
# Como sabemos que encoding tiene un archivo:



def detectar_encoding_csv_url(url):
    """
    Detecta la codificación de un archivo CSV a partir de una URL.

    Args:
        url (str): La URL del archivo CSV.

    Returns:
        str: La codificación detectada (ej. 'utf-8', 'latin-1'), o None si no se pudo detectar.
    """
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Lanza una excepción para errores HTTP
        enc = chardet.detect(response.content[:1024])  # Lee los primeros 1024 bytes
        return enc['encoding']
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar el archivo: {e}")
        return None
    except Exception as e:
        print(f"Error al detectar la codificación: {e}")
        return None



In [4]:
'''# Bloque de codigo encargado de cargar los datos a partir de una url de github y ponerlos en un dataframe


# URL del archivo de texto
url = 'https://github.com/Jorge-Alvarez-23/Data-Science-III_Coder/raw/main/spam.csv'

# Hacer la solicitud GET para obtener el contenido del archivo
response = requests.get(url)

# Verificar que la solicitud fue exitosa
if response.status_code == 200:
    # Leer el contenido CSV directamente en un DataFrame
    datos = StringIO(response.text)  # Convertir el texto a un objeto similar a un archivo
    df = pd.read_csv(datos)  # Leer el contenido en un DataFrame

    # Establecer opción para mostrar todas las columnas
    pd.set_option('display.max_columns', None)  # Muestra todas las columnas
    pd.set_option('display.expand_frame_repr', False)  # Evita que se divida el DataFrame en múltiples líneas

    # Mostrar el DataFrame
    display(df.sample(10))
else:
    print(f'Error al acceder al archivo: {response.status_code}')'''

"# Bloque de codigo encargado de cargar los datos a partir de una url de github y ponerlos en un dataframe\n\n\n# URL del archivo de texto\nurl = 'https://github.com/Jorge-Alvarez-23/Data-Science-III_Coder/raw/main/spam.csv'\n\n# Hacer la solicitud GET para obtener el contenido del archivo\nresponse = requests.get(url)\n\n# Verificar que la solicitud fue exitosa\nif response.status_code == 200:\n    # Leer el contenido CSV directamente en un DataFrame\n    datos = StringIO(response.text)  # Convertir el texto a un objeto similar a un archivo\n    df = pd.read_csv(datos)  # Leer el contenido en un DataFrame\n\n    # Establecer opción para mostrar todas las columnas\n    pd.set_option('display.max_columns', None)  # Muestra todas las columnas\n    pd.set_option('display.expand_frame_repr', False)  # Evita que se divida el DataFrame en múltiples líneas\n\n    # Mostrar el DataFrame\n    display(df.sample(10))\nelse:\n    print(f'Error al acceder al archivo: {response.status_code}')"

In [5]:

import gdown
# OPCIÓN 2: Utilizamos gdown para que cada usuario que acceda a la notebook acceda a los datos sin necesidad de subir el dataset manualmente a su dispositivo.

url = "https://github.com/Jorge-Alvarez-23/Data-Science-III_Coder/raw/main/spam.csv"  # Reemplaza con tu URL
encoding = detectar_encoding_csv_url(url)

# Descargamos el archivo con gdown
gdown.download(url, 'Spam.csv', quiet=False)

# Leemos el archivo CSV descargado
df = pd.read_csv('Spam.csv', encoding=encoding)

# Mostramos las primeras filas

print(df.head(10))

# Mostrar la codificación detectada
print("\n---")
print(f"La codificación detectada es: {encoding}")

Downloading...
From: https://github.com/Jorge-Alvarez-23/Data-Science-III_Coder/raw/main/spam.csv
To: /content/Spam.csv
100%|██████████| 504k/504k [00:00<00:00, 11.1MB/s]

     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   
5  spam  FreeMsg Hey there darling it's been 3 week's n...        NaN   
6   ham  Even my brother is not like to speak with me. ...        NaN   
7   ham  As per your request 'Melle Melle (Oru Minnamin...        NaN   
8  spam  WINNER!! As a valued network customer you have...        NaN   
9  spam  Had your mobile 11 months or more? U R entitle...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  
5        NaN        NaN  
6        NaN  

In [6]:
# Reduciremos el df utilizando solo las columnas que nos interesan.

df = df[["v1", "v2"]]

In [7]:
# Visualizamos algunas de las reviews.

# La número 49
print(df.loc[50, 'v2'])
print("\n" + "-"*80 + "\n")  # línea separadora


# La número 300
print(df.loc[301, 'v2'])
print("\n" + "-"*80 + "\n")  # línea separadora


# La número 5000
print(df.loc[5001, 'v2'])
print("\n" + "-"*80 + "\n")  # línea separadora


What you thinked about me. First time you saw me in class.

--------------------------------------------------------------------------------

Shit that is really shocking and scary, cant imagine for a second. Def up for night out. Do u think there is somewhere i could crash for night, save on taxi?

--------------------------------------------------------------------------------

You still around? Looking to pick up later

--------------------------------------------------------------------------------



In [8]:
df.head(10)

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


# Normalizamos los valores pasando todo el texto a minúscula.

In [9]:
#  ¿Qué hace .lower()? Convierte todo el texto a minúsculas.

# No necesitás importar ninguna librería para usar .lower(), ya que es un método incorporado en los objetos de tipo str en Python.
# Esto significa que cualquier texto (cadena de caracteres) puede convertirse a minúsculas directamente con .lower().
#  Es útil para normalizar texto antes de analizarlo o procesarlo.

df['v2_lower'] = df['v2'].apply(lambda text: text.lower())


In [10]:
# previsualizamos ambas columnas de review para obtener una comparativa.
df

,v1,v2,v2_lower
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ..."
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro..."
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,this is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?,will ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s...","pity, * was in mood for that. so...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...,the guy did some bitching but i acted like i'd...


# Quitamos signos de puntuación

In [11]:
import re

# Definir una función que remueva los signos de puntuación usando re
def remove_punctuation(text):
    # Expresión regular para reemplazar cualquier signo de puntuación o símbolo no alfanumérico
     return re.sub(r'[^a-zA-Z0-9\s]', '', text)

# Aplicar la función a la columna 'review_lower'
df['v2_puntuacion'] = df['v2_lower'].apply(remove_punctuation)

In [12]:
# Previsualizamos algunos ejemplos
df.head(50)

,v1,v2,v2_lower,v2_puntuacion
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ...",go until jurong point crazy available only in ...
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...,u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro...",nah i dont think he goes to usf he lives aroun...
5,spam,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey there darling it's been 3 week's n...,freemsg hey there darling its been 3 weeks now...
6,ham,Even my brother is not like to speak with me. ...,even my brother is not like to speak with me. ...,even my brother is not like to speak with me t...
7,ham,As per your request 'Melle Melle (Oru Minnamin...,as per your request 'melle melle (oru minnamin...,as per your request melle melle oru minnaminun...
8,spam,WINNER!! As a valued network customer you have...,winner!! as a valued network customer you have...,winner as a valued network customer you have b...
9,spam,Had your mobile 11 months or more? U R entitle...,had your mobile 11 months or more? u r entitle...,had your mobile 11 months or more u r entitled...
